In [4]:
import pandas as pd
import numpy as np

import spacy

In [5]:
train = pd.read_csv('data/train.csv')
#test = pd.read_csv('data/test.csv')

In [6]:
print(train.shape)

(26799, 2)


# Check what targets we have

In [7]:
train.Label.unique()

array(['add_to_playlist', 'search_screening_event', 'book_restaurant',
       'rate_book', 'get_weather', 'play_music', 'search_creative_work',
       'no_intent'], dtype=object)

In [8]:
train.Label.value_counts()

no_intent                 12645
get_weather                2041
play_music                 2034
book_restaurant            2029
search_screening_event     2018
search_creative_work       2014
rate_book                  2009
add_to_playlist            2009
Name: Label, dtype: int64

In [32]:
train.loc[26774].Request

'Quer ir para um lugar apropriado para nos pegarmos?'

# Check for missing data

In [9]:
train.isnull().sum()

Request    272
Label        0
dtype: int64

# Drop missings

In [12]:
train = train.dropna()

# Look at unique words (lower case)

In [26]:
cc = CountVectorizer().fit(train['Request'])

In [27]:
cc.get_feature_names()

['00',
 '000',
 '003½',
 '006',
 '007',
 '00h',
 '00s',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '0h',
 '0h00',
 '0i',
 '10',
 '100',
 '1000',
 '101',
 '103rd',
 '105th',
 '1098',
 '10h',
 '10h31',
 '10th',
 '11',
 '116th',
 '117',
 '119',
 '11h',
 '12',
 '120',
 '125',
 '129',
 '12h',
 '12h00',
 '12º',
 '13',
 '131',
 '135th',
 '138th',
 '13h',
 '14',
 '141',
 '144',
 '145',
 '14h',
 '15',
 '1500',
 '1525',
 '157',
 '16',
 '163',
 '1634',
 '165',
 '16h',
 '17',
 '170',
 '1715',
 '1737',
 '175',
 '18',
 '180',
 '1814',
 '183',
 '1844',
 '19',
 '190',
 '1900',
 '1933',
 '1938',
 '1942',
 '1945',
 '1950',
 '1951',
 '1952',
 '1953',
 '1954',
 '1955',
 '1956',
 '1957',
 '1958',
 '1959',
 '1960',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991

# Turn everythin into lower case

train.Request = train.Request.str.lower()
test.Request = test.Request.str.lower()

# Drop missings

In [ ]:
train = train.dropna()

# Music

In [13]:
music_idx = (train.Label == 'play_music') | (train.Label == 'add_to_playlist')

In [14]:
train[train.Request.str.contains('Adicionar')].Label.value_counts()

add_to_playlist         466
no_intent                23
rate_book                 3
play_music                1
search_creative_work      1
Name: Label, dtype: int64

In [16]:
train[(train.Label == 'add_to_playlist')].head()

,Request,Label
0,Adicionar música tomohisa yamashita à minha li...,add_to_playlist
1,Eu quero adicionando Aprite le finestre à minh...,add_to_playlist
7,Eu quero adicionar uma música de wc clark para...,add_to_playlist
11,Adicione a música à de reprodução do Metal.,add_to_playlist
31,colocou o artista Emre Aydin na playlist Happy...,add_to_playlist


### Things with 'música' that aren't related to 'play_music' or 'add_to_playlist'

In [17]:
not_music_idx = ~((train.Label == 'play_music') | (train.Label == 'add_to_playlist'))

In [18]:
not_related_to_music = train[not_music_idx]

In [19]:
not_related_to_music.Request.str.contains('música').sum()

283

In [20]:
not_related_to_music.Request.str.contains('musica').sum()

4

# TODO: restaurant

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
is_book_restaurant = (train.Label == 'book_restaurant')

In [23]:
counter = CountVectorizer()

In [24]:
counter.fit(train[is_book_restaurant].Request)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [ ]:
def expand_with_meta_features(df, text_col, stopwords):
    df = df.copy()
    stop_words = set(stopwords)
    
    df['words'] = df[text_col].str.split(' ').map(len)
    df['words_not_stopword'] = df[text_col].apply(
        lambda x: len([t for t in x.split(' ') if t not in stop_words]))
    df['commas'] = df[text_col].str.count(',')
    df['upper'] = df[text_col].map(lambda x: map(str.isupper, x)).map(sum)
    df['capitalized'] = df[text_col].map(lambda x: map(str.istitle, x)).map(sum)
    df['avg_word_length'] = df[text_col].apply(
        lambda x: np.mean([len(t) for t in x.split(' ') 
                           if t not in stop_words]) 
        if len([len(t) for t in x.split(' ') 
                if t not in stop_words]) > 0 else 0)
    
    return df

In [ ]:
regex_list = [
    ('umidade', 'humidade'), 
    (r'\búltim\b', 'último')
]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
cv.fit(train.Request)

In [ ]:
sorted(cv.vocabulary_.items(), key=lambda e: -e[1])

In [ ]:
train[train.Request.str.contains(r'\búltim\b')]

# spaCy

In [ ]:
from collections import Counter

In [ ]:
#nlp = spacy.load('pt_core_news_sm')
nlp = spacy.load('xx_ent_wiki_sm')

In [ ]:
docs = list(nlp.pipe(train.Request, n_threads=8))

In [ ]:
rows_with_entities = []
rows_without_entities = []
all_entities = []

for i, doc in enumerate(docs):
    entities = doc.ents
    if len(entities) == 0:
        rows_without_entities.append(i)
    else: 
        rows_with_entities.append(i)
        
for i in rows_with_entities:
    all_entities += list(docs[i].ents)

In [ ]:
all_unique_entities = list(
    set(
        list(
            sorted(
                map(lambda e: (str(e.as_doc()), e.label_), all_entities), 
                key=lambda e: e[0]))))

In [ ]:
sorted(all_unique_entities, key=lambda e: e[0])

In [ ]:
cc = Counter(map(lambda e: (str(e.as_doc()), e.label_), all_entities))

In [ ]:
entidades_sem_sentido = [
    ('Adicionar', ''), 
    ('Vamos', ''), 
    ('Preciso', ''), 
    ('Diga', ''), 
    ('Avalie', ''), 
    ('Mostre', ''), 
    ('Reproduzir', ''), 
    ('Mostre', ''), 
    ('Precisa', ''), 
    ('Adicionar', ''), 
    ('Olhe', ''), 
    ('Quero', ''), 
    ('Encontre', ''), 
    ('Você', ''), 
    ('Vamos falar ', '')
]

In [ ]:
ner_df = pd.DataFrame(cc.most_common())

In [ ]:
ner

In [ ]:
z = docs[30]

In [ ]:
for token in z:
    print(token.text, token.pos_)

In [ ]:
g = nlp("eu sou o Bruno")

In [ ]:
for token in g:
    print(token.text, token.pos_)

# Experiências com regex

In [ ]:
s = "esta frase tem uma pessoa chamada Bruno Dias e outra chamada Silva Dias"

In [ ]:
p = r"([A-Z][a-z]+(?=\s[A-Z])(?:\s[A-Z][a-z]+)+)"

In [ ]:
re.findall(p, s)